In [34]:
##IMPORTS
from __future__ import division
import rhinoinside
rhinoinside.load()
import System
import Rhino.Geometry as rg
from Rhino.Geometry import Point3d, Plane, Brep, Vector3d
from Rhino.Geometry import Surface as RhinoSurface
from Rhino.Geometry import Polyline
from Rhino.Collections import Point3dList
from Rhino.Geometry import PolylineCurve
from Rhino.Geometry import Transform
from Rhino.FileIO import File3dm
import Rhino
import Rhino.UI as rui
import ladybug.epw as epw

from ladybug_geometry.geometry2d.pointvector import Vector2D

from lbt_recipes.version import check_radiance_date

#IMPORTS FOR INCIDENT RADIATION
from ladybug.viewsphere import view_sphere
from ladybug.graphic import GraphicContainer
from ladybug.legend import LegendParameters
from ladybug.color import Colorset

from ladybug_rhino.config import conversion_to_meters
from ladybug_rhino.togeometry import to_joined_gridded_mesh3d, to_vector2d
from ladybug_rhino.fromgeometry import from_mesh3d, from_point3d, from_vector3d
from ladybug_rhino.intersect import intersect_mesh_rays, join_geometry_to_mesh

import ladybug.analysisperiod as ap

from ladybug_radiance.skymatrix import SkyMatrix
import ladybug.analysisperiod as ap

from datetime import datetime, timedelta
import collections
import array
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# check the installed Radiance date and get the path to the gemdaymtx executable
check_radiance_date()

In [35]:
#FONCTIONS GH
def change_model_units_to_meters():
    doc = Rhino.RhinoDoc.ActiveDoc
    
    if doc is None:
        print("Aucun document Rhino actif trouvé.")
        return
    
    model_unit_system = doc.ModelUnitSystem
    if model_unit_system != Rhino.UnitSystem.Meters:
        doc.ModelUnitSystem = Rhino.UnitSystem.Meters
        doc.Views.Redraw()
        
def objectify_output(object_name, output_data):
    """Wrap data into a single custom Python object that can later be de-serialized.

    This is meant to address the same issue as the wrap_output method but it does
    so by simply hiding the individual items from the Grasshopper UI within a custom
    parent object that other components can accept as input and de-objectify to
    get access to the data. This strategy is also useful for the case of standard
    object types like integers where the large number of data points slows down
    the Grasshopper UI when they are output.

    Args:
        object_name: Text for the name of the custom object that will wrap the data.
            This is how the object will display in the Grasshopper UI.
        output_data: A list of data to be stored under the data property of
            the output object.
    """
    class Objectifier(object):
        """Generic class for objectifying data."""

        def __init__(self, name, data):
            self.name = name
            self.data = data

        def ToString(self):
            return '{} ({} items)'.format(self.name, len(self.data))

    return Objectifier(object_name, output_data)

def de_objectify_output(objectified_data):
    """Extract the data from an object that was output from the objectify_output method.

    Args:
        objectified_data: An object that has been output from the objectify_output
            method for which data will be returned.
    """
    return objectified_data.data

def text_objects(text, plane, height, font='Arial',
                 horizontal_alignment=0, vertical_alignment=5):
    """Generate a Bake-able Grasshopper text object from a text string and ladybug Plane.

    Args:
        text: A text string to be converted to a a Grasshopper text object.
        plane: A Ladybug Plane object to locate and orient the text in the Rhino scene.
        height: A number for the height of the text in the Rhino scene.
        font: An optional text string for the font in which to draw the text.
        horizontal_alignment: An optional integer to specify the horizontal alignment
             of the text. Choose from: (0 = Left, 1 = Center, 2 = Right)
        vertical_alignment: An optional integer to specify the vertical alignment
             of the text. Choose from: (0 = Top, 1 = MiddleOfTop, 2 = BottomOfTop,
             3 = Middle, 4 = MiddleOfBottom, 5 = Bottom, 6 = BottomOfBoundingBox)
    """
    txt = Rhino.Display.Text3d(text, from_plane(plane), height)
    txt.FontFace = font
    txt.HorizontalAlignment = AlignmentTypes.horizontal(horizontal_alignment)
    txt.VerticalAlignment = AlignmentTypes.vertical(vertical_alignment)
    return TextGoo(txt)

In [36]:
#FONCTIONS GEOMETRIE
def create_surface(x, y, width, length):
    """
    Create a simple NURBS Rhino Surface, from one of its four angles and its dimensions (width, length)

    Args :
    x (float) : X-coordinate of origin angle
    y (float) : Y-coordinate of origin angle
    width (float) : width of the surface
    length (float) : length of the surface

    Return :
    surface (Rhino.Geometry.NurbsSurface) : Surface created
    """
    pt1 = rg.Point3d(x, y, 0)
    pt2 = rg.Point3d(x, y + width, 0)
    pt3 = rg.Point3d(x + length, y + width, 0)
    pt4 = rg.Point3d(x + length, y, 0)
    
    # Create a point list
    points = [pt1, pt2, pt3, pt4]

    # Create a surface from the previous point list, and return it
    surface = rg.NurbsSurface.CreateFromCorners(pt1, pt2, pt3, pt4)

    return surface
    
def create_panel_grid(grid_size, nb_rangs, nb_pvp_rangs, width, length, height, column_spacing, row_spacing, angle_orientation, angle_variable, semi_transp = False, bande = None, void_space = None):
    """
    According to the parameters, creates a grid of surfaces, later converted into Breps, representing the solar panels.

    Args :
    nb_rangs (int) : Number of PVP per rows
    nb_pvp_rangs (int) : Number of rows
    width (float) : Width of a PVP
    length (float) : Length of a PVP
    column_spacing (float) : Distance between two PVP rows (forced value (= ENTRAXE))
    row_spacing (float) : Distance between two PVP inside a row (can be equals to 0, forced value (=grid_size/nb_rangs))
    angle_orientation (int) : Difference between the field axis (PVP supports' axis) and the North-South axis
    angle_variable (int) : tilt of the PVPs
    semi_transp (bool) : False = classic PV Panel, one surface
                         True = Semi-transparent PV panel, composed by 4 strips of PV cells and 3 strips of space inbetween the cells.
    bande (float) : width of the PV cells strip, in case of semi transp panel
    void_space (float) : width of the void between PV cells strip, in case of semi transp panel 
    
    Return :
    final_brep (Rhino.Geometry.Brep) : Brep containing all the PVPs

    """
    final_brep = rg.Brep()

    for i in range(nb_rangs):
        for j in range(nb_pvp_rangs):
            # (x;y) are the coordinate of the current panel created
            x = (j * column_spacing) - grid_size / 2
            y = (i * row_spacing) - grid_size / 2
            surface = []

            if semi_transp == True:
                for h in range(0,4) :
                    strip_x = x + h * (bande + void_space)
                    surface.append(create_surface(strip_x, y, length, bande))
            else:    
                surface.append(create_surface(x, y, length, width))
                
            # Rotation around the X axis for each Brep individually ...
            center_of_surface = Point3d(x + length / 2, y + width / 2, 0)
            rotation_x = Transform.Rotation(angle_variable * (System.Math.PI / 180), Vector3d.YAxis, center_of_surface)
            for surf in surface:
                surf.Transform(rotation_x)
                # ... and add it to the returned brep
                final_brep.Append(surf.ToBrep())       
 
    # Global rotation around the Z axis at the center of the grid (origin)
    rotation_z = Transform.Rotation(angle_orientation * (System.Math.PI / 180), Vector3d.ZAxis, rg.Point3d.Origin)
    final_brep.Transform(rotation_z)

    # Global translation on the Z axis, equivalent to the panel rotation axis height
    translation_z = Transform.Translation(0,0,height)
    final_brep.Transform(translation_z)

    # Inversion of the orientation of the normals for proper operation of the Incident Radiation function
    final_brep.Flip()

    return final_brep

def create_sensor_grid_2(land_orientation, x, y, culture = False):
    """
    Creates the surface from which the Incident Radiation will measure the ground irradiance.

    Args :
    ANGLE_ORIENTATION (int) : Difference between the field axis (PVP supports' axis) and the North-South axis
    culture (bool): If True, the measure surface will correspond to a rectangle that will fit under an arboriculture solar panel (used in vine for now)
                    If False, it will correspond to a square, that we could use in field crops.
    
    Return:
    sensor_brep_list (list(Rhino.Geometry.Brep)) : Surface created from create_surface function, then converted to a brep, then rotated and translated
                                                    if in the case of arboriculture.

    """
    brep_gen = rg.Brep()
    sensor_brep_list = []
    TAILLE_SENSOR = 5 #Sensor side length = 5 mètres
    
    if culture == True:
        # Creating a measure grid that fits under an arboriculture PVP (irradiance is not interesting in between vine rows)
        s = create_surface(x,y,5,0.75) #Create a surface at the origin, length = 5 and width = 0.75
        brep_gen.Append(s.ToBrep())
        
        rotation_z = rg.Transform.Rotation(land_orientation * (System.Math.PI / 180), Vector3d.ZAxis, rg.Point3d.Origin)
        # Translation sous les panneaux /!\ A automatiser pour que ça se mettre automatiquement sous les panneaux

        # Creating Transformations (Rhino.Geometry.Transform)
        v = Vector3d(1,0,0)
        v.Transform(rotation_z)
        translation = rg.Transform.Translation(v * 0.45)
        # Applying Transformations to the measure grid.
        brep_gen.Transform(rotation_z)
        brep_gen.Transform(translation)
    

    if culture == False:
        #Create a square with TAILLE_SENSOR is the size of a side, center of the square is the origin of the Rhino Environment
        surface_brep = create_surface(-TAILLE_SENSOR / 2, -TAILLE_SENSOR / 2 , TAILLE_SENSOR, TAILLE_SENSOR)
        brep_gen.Append(surface_brep.ToBrep())
    
    brep_gen.Flip()
    sensor_brep_list.append(brep_gen)

    return sensor_brep_list

def export_brep_to_file(brep, filename, new_file = True):
    """
    Creates a 3dm file to visualize the objects, mainly Breps, created wiht python scripts.

    Args:
    brep (Rhino.Geometry.Brep): Brep to visualize
    filename (str): Name of the output file.

    Return:
    Nothing, the file is created

    """
    if new_file == False:
        file = File3dm.Read(filename)
        if brep:
            file.Objects.AddBrep(brep)
            file.Write(filename,7)  # Rewrite the file with the new geometry
            print("La nouvelle géométrie a été ajoutée au fichier existant.")
        else:
            print("Erreur lors de la création de la surface.")

    else :
        file = File3dm()
        file.Settings.ModelUnitSystem = Rhino.UnitSystem.Meters
        file.Objects.AddBrep(brep)
        file.Write(filename, 7)  # 7 corresponding to Rhino 7

In [37]:
#FONCTIONS RADIANCE
def create_sky_matrix(_location, _direct_rad, _diffuse_rad, _folder_ = None, north_ = None, _hoys_ = None):
    """
    Get a matrix containing radiation values from each patch of a sky dome.
    _
    Creating this matrix is a necessary pre-step before doing incident radiation
    analysis with Rhino geometry or generating a radiation rose.
    _
    This component uses Radiance's gendaymtx function to calculate the radiation
    for each patch of the sky. Gendaymtx is written by Ian Ashdown and Greg Ward.
    Morere information can be found in Radiance manual at:
    http://www.radiance-online.org/learning/documentation/manual-pages/pdfs/gendaymtx.pdf
    -

    Args:
        north_: A number between -360 and 360 for the counterclockwise
            difference between the North and the positive Y-axis in degrees.
            90 is West and 270 is East. This can also be Vector for the
            direction to North. (Default: 0)
        _location: A ladybug Location that has been output from the "LB Import EPW"
            component or the "LB Construct Location" component.
        _direct_rad: An annual hourly DataCollection of Direct Normal Radiation such
            as that which is output from the "LB Import EPW" component or the
            "LB Import STAT" component.
        _diffuse_rad: An annual hourly DataCollection of Diffuse Horizontal Radiation
            such as that which is output from the "LB Import EPW" component or
            the "LB Import STAT" component.
        _hoys_: A number or list of numbers between 0 and 8760 that respresent
            the hour(s) of the year for which to generate the sky matrix. The
            "LB Calculate HOY" component can output this number given a month,
            day and hour. The "LB Analysis Period" component can output a
            list of HOYs within a certain hour or date range. By default,
            the matrix will be for the entire year.
        high_density_: A Boolean to indicate whether the higher-density Reinhart sky
            matrix should be generated (True), which has roughly 4 times the sky
            patches as the (default) original Tregenza sky (False). Note that,
            while the Reinhart sky has a higher resolution and is more accurate,
            it will result in considerably longer calculation time for incident
            radiation studies. The difference in sky resolution can be observed
            with the "LB Sky Dome" component. (Default: False).
        _ground_ref_: A number between 0 and 1 to note the average ground reflectance
            that is associated with the sky matrix. (Default: 0.2).
        _folder_: The folder in which the Radiance commands are executed to
            produce the sky matrix. If None, it will be written to Ladybug's
            default EPW folder.

    Returns:
        report: ...
        sky_mtx: A sky matrix object containing the radiation coming from each patch
            of the sky. This can be used for a radiation study, a radition rose,
            or a sky dome visualization. It can also be deconstructed into its
            individual values with the "LB Deconstruct Matrix" component.
    """
    if north_ is not None:  # process the north_
        try:
            north_ = math.degrees(
                to_vector2d(north_).angle_clockwise(Vector2D(0, 1)))
        except AttributeError:  # north angle instead of vector
            north_ = float(north_)
    if _hoys_ is not None: # process if hoys are specified or not
        sky_mtx = SkyMatrix.from_components(
            _location, _direct_rad, _diffuse_rad, 
            _hoys_) #, north_, high_density_, ground_r
        if _folder_:
            sky_mtx.folder = _folder_
    else:
        # create the sky matrix object
        sky_mtx = SkyMatrix.from_components(
            _location, _direct_rad, _diffuse_rad, ) #_hoys_, north_, high_density_, ground_r  
        if _folder_:
            sky_mtx.folder = _folder_

    return sky_mtx

def incident_radiation(_sky_mtx_in_IR, _geometry, context_, _offset_dist_, _finesse = 1, irradiance_ = True, legend_par_ = None):
    """
    Calculate the incident radiation on geometry using a sky matrix from the "Cumulative
    Sky Matrix" component.
    _
    Such studies of incident radiation can be used to apprxomiate the energy that can
    be collected from photovoltaic or solar thermal systems. They are also useful
    for evaluating the impact of a building's orientation on both energy use and the
    size/cost of cooling systems. For studies of photovoltaic potential or building
    energy use impact, a sky matrix from EPW radiation should be used. For studies
    of cooling system size/cost, a sky matrix derived from the STAT file's clear sky
    radiation should be used.
    _
    NOTE THAT NO REFLECTIONS OF SOLAR ENERGY ARE INCLUDED
    IN THE ANALYSIS PERFORMED BY THIS COMPONENT.
    _
    Ground reflected irradiance is crudely acounted for by means of an emissive
    "ground hemisphere," which is like the sky dome hemisphere and is derived from
    the ground reflectance that is associated with the connected _sky_mtx. This
    means that including geometry that represents the ground surface will effectively
    block such crude ground reflection.
    _
    Also note that this component uses the CAD environment's ray intersection methods,
    which can be fast for geometries with low complexity but does not scale well for
    complex geometries or many test points. For such complex cases and situations
    where relfection of solar energy are important, honeybee-radiance should be used.
    -

    Args:
        _sky_mtx_in_IR: A Sky Matrix from the "LB Cumulative Sky Matrix" component or the
            "LB Benefit Sky Matrix" component, which describes the radiation
            coming from the various patches of the sky. The "LB Sky Dome"
            component can be used to visualize any sky matrix to understand
            its relationship to the test geometry.
        _geometry: Rhino Breps and/or Rhino Meshes for which incident radiation analysis
            will be conducted. If Breps are input, they will be subdivided using
            the _grid_size to yeild individual points at which analysis will
            occur. If a Mesh is input, radiation analysis analysis will be
            performed for each face of this mesh instead of subdividing it.
        context_: Rhino Breps and/or Rhino Meshes representing context geometry
            that can block solar radiation to the test _geometry.
        _offset_dist_: A number for the distance to move points from the surfaces
            of the input _geometry.  Typically, this should be a small positive
            number to ensure points are not blocked by the mesh. (Default: 10 cm
            in the equivalent Rhino Model units).
        _finesse: A positive number in Rhino model units for the size of grid
            cells at which the input _geometry will be subdivided for incident
            radiation analysis. The smaller the grid size, the higher the
            resolution of the analysis and the longer the calculation will take.
            So it is recommended that one start with a large value here and
            decrease the value as needed. However, the grid size should usually
            be smaller than the dimensions of the smallest piece of the _geometry
            and context_ in order to yield meaningful results.
        irradiance_: Boolean to note whether the study should output units of cumulative
            Radiation (kWh/m2) [False] or units of average Irradiance (W/m2)
            [True].  (Default: False).
        legend_par_: Optional legend parameters from the "LB Legend Parameters"
            that will be used to customize the display of the results.
        _cpu_count_: An integer to set the number of CPUs used in the execution of the
            intersection calculation. If unspecified, it will automatically default
            to one less than the number of CPUs currently available on the
            machine or 1 if only one processor is available.
        _run: Set to "True" to run the component and perform incident radiation
            analysis.

    Returns:
        report: ...
        points: The grid of points on the test _geometry that are be used to perform
            the incident radiation analysis.
        results: A list of numbers that aligns with the points. Each number indicates
            the cumulative incident radiation received by each of the points
            from the sky matrix in kWh/m2.
        total: A number for the total incident solar energy falling on all input geometry
            in kWh. Note that, unlike the radiation results above, which are
            normlaized by area, these values are not area-normalized and so
            the input geometry must be represented correctly in the Rhino
            model's unit system in order for this output to be meaningful.
        mesh: A colored mesh of the test _geometry representing the cumulative
            incident radiation received by the input _geometry.
        legend: A legend showing the kWh/m2 that correspond to the colors of the mesh.
        title: A text object for the study title.
        int_mtx_to_RTIR: A Matrix object that can be connected to the "LB Deconstruct Matrix"
            component to obtain detailed patch-by-patch results of the study.
            Each sub-list of the matrix (aka. branch of the Data Tree) represents
            one of the points used for analysis. The length of each sub-list
            matches the number of sky patches in the input sky matrix (145 for
            the default Tregenza sky and 577 for the high_density Reinhart sky).
            Each value in the sub-list is a value between 0 and 1 indicating the
            relationship between the point and the patch of the sky. A value of
            "0", indicates that the patch is not visible for that point at all
            while a value of "1" indicates that the patch hits the surface that
            the point represents head on.
    """

    # set the default offset distance and _cpu_count
    _offset_dist_ = _offset_dist_ if _offset_dist_ is not None \
        else 0.5 / conversion_to_meters()
    workers = 1 # _cpu_count_ if _cpu_count_ is not None else recommended_processor_count()

    # create the gridded mesh from the geometry
    study_mesh = to_joined_gridded_mesh3d(_geometry, _finesse)
    points = [from_point3d(pt.move(vec * _offset_dist_)) for pt, vec in
            zip(study_mesh.face_centroids, study_mesh.face_normals)]

    # mesh the geometry and context
    shade_mesh = join_geometry_to_mesh(_geometry + [context_])

    # deconstruct the matrix and get the sky dome vectors
    mtx_temp_IR = de_objectify_output(_sky_mtx_in_IR)
    total_sky_rad = [dir_rad + dif_rad for dir_rad, dif_rad in zip(mtx_temp_IR[1], mtx_temp_IR[2])]
    ground_rad = [(sum(total_sky_rad) / len(total_sky_rad)) * mtx_temp_IR[0][1]] * len(total_sky_rad)
    all_rad = total_sky_rad + ground_rad 
    lb_vecs = view_sphere.tregenza_dome_vectors if len(total_sky_rad) == 145 \
        else view_sphere.reinhart_dome_vectors
    if mtx_temp_IR[0][0] != 0:  # there is a north input for sky; rotate vectors
        north_angle = math.radians(mtx_temp_IR[0][0])
        lb_vecs = tuple(vec.rotate_xy(north_angle) for vec in lb_vecs)
    lb_grnd_vecs = tuple(vec.reverse() for vec in lb_vecs)
    all_vecs = [from_vector3d(vec) for vec in lb_vecs + lb_grnd_vecs]

    # intersect the rays with the mesh
    normals = [from_vector3d(vec) for vec in study_mesh.face_normals]
    int_matrix_init, angles = intersect_mesh_rays(
        shade_mesh, points, all_vecs, normals, cpu_count=workers)

    # compute the results
    results = []
    int_matrix = []
    for int_vals, angs in zip(int_matrix_init, angles):
        pt_rel = [ival * math.cos(ang) for ival, ang in zip(int_vals, angs)]
        int_matrix.append(pt_rel)
        rad_result = sum(r * w for r, w in zip(pt_rel, all_rad))
        results.append(rad_result)

    # convert to irradiance if requested -> NOT DONE
    if irradiance_:
        factor = 1000 / _sky_mtx_in_IR.wea_duration if hasattr(_sky_mtx_in_IR, 'wea_duration') \
            else 1000 / (((mtx_temp_IR[0][3] - mtx_temp_IR[0][2]).total_seconds() / 3600) + 1)
        results = [r * factor for r in results]

    # output the intersection matrix and compute total radiation
    int_mtx_to_RTIR = objectify_output('Geometry/Sky Intersection Matrix', int_matrix)
    unit_conv = conversion_to_meters() ** 2
    total = 0
    for rad, area in zip(results, study_mesh.face_areas):
        total += rad * area * unit_conv
    return int_mtx_to_RTIR

Paramétrage :

In [38]:
# Paramètres de configuration de la géométrie
######
GRID_SIZE = 50 # Taille du champ de panneau pour la simulation
ENTRAXE = 2.5 # Espacement entre lignes de panneaux (= inter-rang)
HAUTEUR = 3 # [0 ; 100] ; mètres ; hauter de l'axe de rotation, ou du point du PVP le plus bas.
RAMPANT = 1.135 # Largeur du panneau
NB_RANGS = 10 # Nombre de panneaux par ligne
NB_PVP_RANGS = int(GRID_SIZE / ENTRAXE) # Nombre de lignes
LONGUEUR_PVP = GRID_SIZE / (NB_RANGS) # Longueur des panneaux dans une même ligne
LARGEUR_BANDE = 0.18
LARGEUR_AVIDE = (RAMPANT-4*LARGEUR_BANDE)/3

column_spacing = ENTRAXE # Espacement entre lignes de panneaux (= inter-rang)
row_spacing = GRID_SIZE / NB_RANGS # Espacement entre panneaux d'une même ligne (même support)

ANGLE_ORIENTATION = 13  # Rotation globale autour de l'axe Z (Orientation parcelle)
ANGLE_VARIABLE = -30  # Angle de rotation individuelle autour de l'axe X

# Paramètres horaires et annuels
######
HEURE_DEB = 12
HEURE_FIN = 13
start_date = datetime(year=2023, month=1, day=1, hour=HEURE_DEB)  # Début de l'année (éviter les années bissextiles)
end_date = datetime(year=2024, month=1, day=1, hour=HEURE_DEB)  # Fin de l'année (début de l'année suivante)
NB_HEURES_ANNEES = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days

# Paramètres angles
#####
start_angle = -60
end_angle = 60
step_angle = 60
angles = np.arange(start_angle, end_angle + 1, step_angle)
print(list(angles))

# Paramètres radiance
#####
SKYMAT_FOLDER = "cielMatrices/"
HAUTEUR_MESURE = 0.1 # Hauteur de mesure dans IncidentRadiation
FINESSE = 1 # Finesse de la fonction IncidentRadiation (1 = peu précis, 0 = très précis)

# Appel de la fonction pour changer les unités du modèle en mètres
#####

[-60, 0, 60]


# TEST SURFACE

In [39]:
# angle2 = 15
# PANNEAUX = create_panel_grid(GRID_SIZE, NB_RANGS, NB_PVP_RANGS, RAMPANT, LONGUEUR_PVP, HAUTEUR, column_spacing, row_spacing, ANGLE_ORIENTATION, angle2, semi_transp=True, bande=LARGEUR_BANDE, void_space=LARGEUR_AVIDE)
# SOL = create_sensor_grid_2(ANGLE_ORIENTATION,0, 0, culture=True)
# export_brep_to_file(PANNEAUX, "output_test.3dm")

# file_path = "output_test.3dm"
# file = File3dm.Read(file_path)
# if SOL:
#     file.Objects.AddBrep(SOL[0])
#     file.Write(file_path,7)  # Réécrire le fichier avec la nouvelle géométrie ajoutée
#     print("La nouvelle géométrie a été ajoutée au fichier existant.")
# else:
#     print("Erreur lors de la création de la surface.")

# TEST SURFACE

Import EPW

In [40]:
# Here, the meteo file (epw format) is imported and necessary data for SkyMatrix construction is extracted
_epw_file_path = "FRA_AC_Agen-La.Garenne.AP.075240_TMYx.epw"
epw_data = epw.EPW(_epw_file_path)

location_ = epw_data.location #######
direct_normal_rad_ = epw_data.direct_normal_radiation #########
diffuse_horizontal_rad_ = epw_data.diffuse_horizontal_radiation #########

Boucle 1 (= main)

In [61]:
# Creating the tab that will store every irradiation value, for every angle, for every hour of the year
tab_hoys_angles = np.zeros((NB_HEURES_ANNEES, len(angles)))

# The measurement grid does not change depending on the inclination of the panels, it can only be created once. True = arboriculture
measures_grid = create_sensor_grid_2(ANGLE_ORIENTATION,0,0,culture=True)

# Crate first skymatrix (full year) to generate full year int_mtx
s1 = create_sky_matrix(location_, direct_normal_rad_, diffuse_horizontal_rad_, SKYMAT_FOLDER)

#Créer géom, int_mtx_TEMOIN et data pour zone témoin (une fois)
panneaux_TEMOIN = rg.Brep()
panneaux_TEMOIN.Append(create_surface(40, 40, 0.1, 0.1).ToBrep())
sol_temoin = create_sensor_grid_2(ANGLE_ORIENTATION,40,40,culture=True)
int_mtx_TEMOIN = np.array(de_objectify_output(incident_radiation(s1, sol_temoin, panneaux_TEMOIN, HAUTEUR_MESURE, FINESSE)), dtype=object)
columns_TEMOIN = np.empty((0, len(int_mtx_TEMOIN)))

#Boucle 1, Varying parameter : = tilt angle of PVPs
for index, angle in enumerate(angles):

    #Create panel geometry
    final_rotated_brep = create_panel_grid(GRID_SIZE, NB_RANGS, NB_PVP_RANGS, RAMPANT, LONGUEUR_PVP, HAUTEUR, column_spacing, row_spacing, ANGLE_ORIENTATION, angle)
    
    #Create int_mtx_angle that will be crossed with every hourly sky_mtx
    int_mtx_angle = np.array(de_objectify_output(incident_radiation(s1, measures_grid, final_rotated_brep, HAUTEUR_MESURE, FINESSE)), dtype=object)
    
    #Setting date
    current_date = start_date

    #Create table which will receive the X measurements for one hour for an angle /!\ PROCESS THIS DATA /!\
    columns = np.empty((0, len(int_mtx_angle)))

    #Create table that will receive the X values ​​for all hours of the year for an angle
    irr_all_hoys_one_angle = np.zeros((NB_HEURES_ANNEES, 1))

    while current_date < end_date:
    ##############
    ###BOUCLE 2### Varying parameter : date
    ##############

        if current_date.hour >= HEURE_FIN: #If current hour > HEURE_FIN, which is the last hour we want data 
            #Skip to next day
            current_date += timedelta(days=1)  #+1 day
            current_date = current_date.replace(hour=HEURE_DEB)  #Set hour to starting hour (HEURE_DEB)
            continue #Skip the rest of the loop
        next_date = current_date + timedelta(hours=1) #next_date = current_date + 1h

        ##Analysis Period##-----------------
        anp = ap.AnalysisPeriod(
            current_date.month, current_date.day, current_date.hour, # STARTING DATES
            next_date.month, next_date.day, next_date.hour, # ENDING DATES
            1) # TIMESTEP PER HOUR (SET TO 1)
        hoys_sm = anp.hoys
        ##Analysis Period##-----------------

        ##Temporary SkyMatrix##-------------
        sky_mtx_boucle = create_sky_matrix(location_,
            direct_normal_rad_,
            diffuse_horizontal_rad_,
            SKYMAT_FOLDER,
            _hoys_=hoys_sm)
        ##Temporary SkyMatrix##-------------

        ##LB RTIR##-------------------------
        sky_mtx_temp_RTIR = np.array(de_objectify_output(sky_mtx_boucle), dtype = object)
        
        total_sky_rad = sky_mtx_temp_RTIR[1] + sky_mtx_temp_RTIR[2]
        average_sky_rad = np.mean(total_sky_rad)
        ground_rad = average_sky_rad * sky_mtx_temp_RTIR[0][1] * np.ones_like(total_sky_rad)
        all_rad = total_sky_rad + ground_rad

        for pt_rel in int_mtx_angle:
            results_RTIR = np.dot(int_mtx_angle, all_rad)
        columns = np.vstack((columns, results_RTIR))

        #Zone Témoin#---
        #Ne s'exécute qu'une fois car pas de structure (context_ dans IncidentRadiation)
        #Zone Témoin#---

        ##LB RTIR##-------------------------
        
        current_date += timedelta(hours=1)  # Incrémenter de une heure
        
        ##############
        ###BOUCLE 2###
        ##############

    irr_all_hoys_one_angle = np.mean(columns, axis=1).reshape(-1, 1)
    tab_hoys_angles[:, index] = irr_all_hoys_one_angle.squeeze()

#-- Data control zone 
for pt_rel2 in int_mtx_TEMOIN:
    results_RTIR_TEMOIN = np.dot(int_mtx_TEMOIN, all_rad)
columns_TEMOIN = np.vstack((columns_TEMOIN, results_RTIR_TEMOIN))
irr_all_hoys_TEMOIN = np.mean(columns, axis=1).reshape(-1, 1)
# tab_hoys_angles_temoin = np.column_stack((tab_hoys_angles, irr_all_hoys_TEMOIN))
# #-- Data control zone/



In [62]:
print(int_mtx_angle)
print(int_mtx_TEMOIN)

[[0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]]
[[0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]
 [0.10396205118449753 0.10396205118449753 0.1039620511844973 ... -0.0
  -0.0 -0.0]]


In [47]:
#VISU
file_path = "output_apres_boucle.3dm"
export_brep_to_file(final_rotated_brep, file_path)
export_brep_to_file(measures_grid[0], file_path, False)
export_brep_to_file(panneaux_TEMOIN, file_path, False)
export_brep_to_file(sol_temoin[0], file_path, False)

La nouvelle géométrie a été ajoutée au fichier existant.
La nouvelle géométrie a été ajoutée au fichier existant.
La nouvelle géométrie a été ajoutée au fichier existant.


In [55]:
df2 = pd.DataFrame(tab_hoys_angles)
df2

,0,1,2
0,0.289001,0.205809,0.259271
1,0.004720,0.003291,0.003972
2,0.509953,0.351490,0.449788
3,0.389421,0.251734,0.335245
4,0.015093,0.010180,0.012629
...,...,...,...
360,0.183226,0.156738,0.182536
361,0.004003,0.002791,0.003369
362,0.081222,0.058003,0.072724
363,0.445482,0.340268,0.416377


In [54]:
# df1 = pd.DataFrame(tab_hoys_angles_temoin)
# df1.columns = np.append(angles, np.array("temoin", dtype=object), axis=None)
df1

,-60,0,60,temoin
0,0.289001,0.205809,0.259271,0.259271
1,0.00472,0.003291,0.003972,0.003972
2,0.509953,0.35149,0.449788,0.449788
3,0.389421,0.251734,0.335245,0.335245
4,0.015093,0.01018,0.012629,0.012629
...,...,...,...,...
360,0.183226,0.156738,0.182536,0.182536
361,0.004003,0.002791,0.003369,0.003369
362,0.081222,0.058003,0.072724,0.072724
363,0.445482,0.340268,0.416377,0.416377


0.139